In [59]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy
from collections import defaultdict
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import json

%matplotlib inline

In [2]:
data = pd.read_csv("hillary-clinton-emails/Emails.csv")

In [3]:
data.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [4]:
data["ExtractedBodyText"][0]

nan

In [5]:
data["RawText"][0]

'UNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nRELEASE IN FULL\nFrom: Sullivan, Jacob J <Sullivan11@state.gov>\nSent: Wednesday, September 12, 2012 10:16 AM\nTo:\nSubject: FW: Wow\nFrom: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi\nSent: Wednesday, September 12, 2012 10:09 AM\nTo: Sullivan, Jacob J\nSubject: Wow\nWhat a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me\ntoo\nUNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247\n\x0c'

**Как видно, в колонке "ExtractedBodyText" содержится далеко не весь текст, необходимый для анализа, поэтому придется парсить его из "RawText", отбрасывая все ненужные вспомогательные символы и слова**

In [6]:
def extract_text(text):
    text = re.sub(r"STATE-\w+", "", text)
    text = re.sub(r"^UNCLASSIFIED.*$", "", text, flags=re.M)
    text = re.sub(r"^PART \w+$", "", text, flags=re.M)
    text = re.sub(r"^RELEASE IN\s{0,1}\w*$", "", text, flags=re.M)
    text = re.sub(r"^From.*$", "", text, flags=re.M)
    text = re.sub(r"^Sent.*$", "", text, flags=re.M)
    text = re.sub(r"^To.*$", "", text, flags=re.M)
    text = re.sub(r"^For.*$", "", text, flags=re.M)
    text = re.sub(r"^Fw.*$", "", text, flags=re.M)
    text = re.sub(r"^Importance.*$", "", text, flags=re.M)
    return "\n".join(re.split(r"\n*", text)).strip()

In [7]:
texts = []
for text in data["RawText"]:
    texts.append(extract_text(text))

**Выясним,	какие	биграммы	чаще	всего	встречаются	в	датасете	**

In [8]:
from nltk.tokenize import word_tokenize
words = [word_tokenize(a) for a in texts]

In [9]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(words)

In [10]:
finder.nbest(bigram_measures.raw_freq, 30)

[('of', 'the'),
 ('in', 'the'),
 (',', 'and'),
 ('.', 'The'),
 ('to', 'the'),
 (',', 'the'),
 ('on', 'the'),
 ('.', 'I'),
 ('.', "''"),
 ('and', 'the'),
 ('for', 'the'),
 ('.', '``'),
 ('that', 'the'),
 (',', "''"),
 (',', 'but'),
 ('with', 'the'),
 ('to', 'be'),
 ('.', 'We'),
 ('at', 'the'),
 ('.', 'In'),
 ('.', 'He'),
 ('will', 'be'),
 ('by', 'the'),
 ('.', 'It'),
 ('http', ':'),
 ('.', 'But'),
 ('is', 'a'),
 ('from', 'the'),
 (',', 'a'),
 (',', 'I')]

** Pointwise Mutual Information**

In [11]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(words)
finder.apply_freq_filter(100)
finder.nbest(bigram_measures.pmi, 30)

[('Sinn', 'Fein'),
 ('08/27/2015', 'Class'),
 ('CONFIDENTIAL', 'Reason'),
 ('OFFICE', 'TIME'),
 ('Human', 'Rights'),
 ('Supreme', 'Court'),
 ('Source', 'Comment'),
 ('Watch', 'Officer'),
 ('2201', 'C'),
 ('per', 'cent'),
 ('Saudi', 'Arabia'),
 ('Northern', 'Ireland'),
 ('Lona', 'Valmoro'),
 ('Private', 'Residence'),
 ('SECRETARY', 'CLINTON'),
 ('Wall', 'Street'),
 ('Jacob', 'J'),
 ('*En', 'route'),
 ('prime', 'minister'),
 ('STAFF', 'MEETING'),
 ('En', 'route'),
 ('IN', 'FULL'),
 ('RELEASE', 'IN'),
 ('Tea', 'Party'),
 ('George', 'W.'),
 ('IN', 'PART'),
 ('Middle', 'East'),
 ('Tony', 'Blair'),
 ('civil', 'society'),
 ('Transitional', 'Council')]

Видно, что в обычных биграммах чаще всего встречаются слова, состоящие из предлогов, артиклей и пунктуационных знаков. При использовании PMI биграммы - в которых слова встречаются чаще, чем по-раздельности.   

**Кластеризация: ** TF-IDF + KMeans

In [13]:
from gensim import matutils

In [14]:
tfidf = TfidfVectorizer(max_features=1000)
X = tfidf.fit_transform(texts)
feature_names = tfidf.get_feature_names()
dictionary = dict([(i, s) for i, s in enumerate(feature_names)])
corpus = matutils.Sparse2Corpus(X)
corpus = matutils.Sparse2Corpus(corpus.sparse.T)

In [15]:
clf = KMeans(n_clusters=8)
clusters = clf.fit_predict(X)

In [16]:
centroids = clf.cluster_centers_.argsort()[:, ::-1]

In [17]:
terms = tfidf.get_feature_names()
for i in range(centroids.shape[0]):
    cluster = str(i)
    for ind in centroids[i, :15]:
        cluster += " " + terms[ind]
    print(cluster)

0 gov state pm 2009 abedin huma 2010 clintonemail am mills cheryl com to re sullivan
1 the to you and we of in is for it on that this will have
2 call to you can do and talk the for tomorrow him he me will if
3 to the you in on ok for is and will release fyi b6 this we
4 202 647 state of the to department secretary office gov assistant and you affairs for
5 the of to and in on said for is that with was he by as
6 pm am 00 office 30 secretary depart room arrive route en 15 45 10 residence
7 the of to and in that is for on with as it this we he


**Кластеризация: ** TF-IDF + MiniBatchKMeans

In [18]:
from sklearn.cluster import MiniBatchKMeans

km = MiniBatchKMeans(n_clusters=10, init='k-means++', n_init=1,
                         init_size=10, batch_size=1000)
km.fit(X)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=10, max_iter=100, max_no_improvement=10, n_clusters=10,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [19]:
centroids = km.cluster_centers_.argsort()[:, ::-1]

In [20]:
terms = tfidf.get_feature_names()
for i in range(centroids.shape[0]):
    cluster = str(i)
    for ind in centroids[i, :15]:
        cluster += " " + terms[ind]
    print(cluster)

0 can time if afternoon service to late really change yes go want the your see
1 authorities president election to from known announced committee led presidential the iran earlier november leader
2 b5 b6 release part in cheryl the to full fyi you am 2009 jake and
3 food saturday afternoon wants move ok forward for cheryl security best meeting the if at
4 ok the budget about tonight reach so could him call now you do calling if
5 pm gov state am abedin 2009 huma 00 2010 office 10 30 clintonemail secretary com
6 event and the part ireland now our agency events of per co side available cdm
7 to the you and for in call on we of will is can it do
8 recently something morning down let my had said who me this for in and to
9 the to and of in that is on for with we this as it you


**Визуализация** LDA

https://github.com/bmabey/pyLDAvis

In [28]:
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim

In [63]:
lda = LdaModel(corpus, num_topics=9, passes=30, id2word=dictionary)

In [64]:
pyLDAvis.enable_notebook()
d = Dictionary.from_corpus(corpus, dictionary)
lda_vis = pyLDAvis.gensim.prepare(lda, corpus, d)
pyLDAvis.save_json(lda_vis, 'lda.json')

In [65]:
with open('lda.json', 'r') as j:
    data_input = json.load(j)

emails_model_data = pyLDAvis.PreparedData(pd.DataFrame(data_input['mdsDat']), pd.DataFrame(data_input['tinfo']), 
                  pd.DataFrame(data_input['token.table']), data_input['R'],
                  data_input['lambda.step'], data_input['plot.opts'], data_input['topic.order'])


pyLDAvis.display(emails_model_data)

**Как оценить интерпретируемость кластеров с помощью асессоров**

Если ставить перед асессором задачу оценить уже готовую кластеризацию - можно предложить ему ознакомиться с ключевыми словами в каждом кластере и обозначить тему. Если он затрудняется сделать это для какого-то кластера, или он объявляет для двух кластеров близкую тему(например, как у 1 и 2 кластера), значит наша кластеризация не очень хорошая. Можно предложить асессору оценивать, насколько два текста из одного класса похожи по тематике (именно тексты, а не ключевые слова), но это потребует гораздо больше времени и оценки могут оказаться более субъективными, чем при просмотре ключевых слов.